Notebook to run the ABM multiple time and collect its output to validate it.

## Content
* Real data loading
* Model runs and output collection

In [1]:
# sbp_payments.xlsx for real values of payments; sbp_payments_all0.xlsx for always null payments (counterfactual).
sbp_payments_path = "./data/sbp_payments.xlsx"

# The ML models used are the ones in the he ones in the folder .abm.ml_model

In [2]:
cd ..

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import csv

from abm.model import SBPAdoption

from abm.custom_transformers import (
    TransformCensusFeatures,
    TransformClimateFeatures,
    TransformSoilFeatures,
    )

In [4]:
%matplotlib inline

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [5]:
# Line necessary to import agents in model, since the cwd of the notebook server is always the base path where you started the 
# server 
import sys
sys.path.append(".\abm")

In [6]:
# How to display floats
pd.options.display.float_format = '{:.6f}'.format
#np.set_printoptions(precision=6)
np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})

In [7]:
# Where to save the figures
PROJECT_ROOT_DIR = "model_validation"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "output")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=False, fig_extension="jpeg", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Real data loading

In [8]:
path_to_adoption_port_real = "./model_validation/data/SBP yearly adoption - Portugal.csv"
yearly_adoption_portugal_real = pd.read_csv(path_to_adoption_port_real, index_col=0)
yearly_adoption_portugal_real.drop(1995, inplace=True)

In [9]:
path_to_adoption_munic_real = "./model_validation/data/SBP yearly adoption - Municipalities.csv"
munic_yearly_adoption_real = pd.read_csv(path_to_adoption_munic_real, index_col=['Municipality', 'Year'])

# Model runs and output collection

In [10]:
n_runs = 25

In [11]:
start_year = 1996 # Included in the simulation
stop_year = 2021 # Not included in the simulation

### Set dataframe and function to collect municipalities yearly adoption

In [12]:
municipalities_data = gpd.read_file("./data/municipalities_shp/shapefile_for_munic_abm.shp")
municipalities_data.rename(columns={'Municipali': 'Municipality'},
                           inplace=True)
municipalities = municipalities_data['Municipality'].values.tolist()

years = np.arange(start_year, stop_year)
munic_yearly_adoption = pd.MultiIndex.from_product([municipalities, years])
munic_yearly_adoption = munic_yearly_adoption.to_frame()
munic_yearly_adoption.drop(munic_yearly_adoption.columns, axis=1, inplace=True)
munic_yearly_adoption.index.rename(['Municipality', 'Year'], inplace=True)

In [13]:
def collect_agent_level_yearly_adoption(abm_model, n_run, df_adoption):
    """
    Function to collect data at the individual agent level
    
    Returns
    -------
    municipalities_adoption : pd DataFrame
        Multiindex DataFrame reporting for each muninicipality and for each year the ratio of SBP adopted
    """
    municipality_agents = abm_model.schedule.agents
    col_name = 'Run ' + str(n_run+1)
    df_adoption.loc[:, col_name] = None
    
    # Create a dictionary mapping to each municipality name the Series of its adoption in each year
    yearly_adoptions = [munic.yearly_adoption for munic in municipality_agents]
    map_dict = dict(zip(municipalities, yearly_adoptions))
    for k, v in map_dict.items():
        map_dict[k] = pd.Series(v)
        map_dict[k].name = col_name # Necessary to use .udpdate() method

    # For each municipality, updates the values of adoption    
    for munic in municipalities:
        df = df_adoption.loc[munic]
        df.update(map_dict[munic])
        df_adoption.loc[munic] = df.values

### Set dataframes to collect yearly adoption and cumulative yearly adoption in Portugal

In [14]:
yearly_adoption_portugal_pred = pd.DataFrame(index=years)
yearly_adoption_portugal_pred.index.rename('Year', inplace=True)

cumulative_adoption_portugal_pred = pd.DataFrame(index=years)
cumulative_adoption_portugal_pred.index.rename('Year', inplace=True)

In [15]:
def collect_data_run(model, n_run):
    # Individual adoption
    collect_agent_level_yearly_adoption(model, n_run, munic_yearly_adoption)
    
    # Aggregate adoption
    aggr_adoption_out = model.datacollector.get_model_vars_dataframe()
    aggr_adoption_out.set_index('Year', inplace=True)
    col_name = 'Run ' + str(n_run+1)
    yearly_adoption_portugal_pred[col_name] = aggr_adoption_out['Yearly area of SBP sown [ha/y]']
    cumulative_adoption_portugal_pred[col_name] = aggr_adoption_out['Cumulative area of SBP sown [ha]']

In [16]:
%%time
for i in range(n_runs):
    model = SBPAdoption(initial_year=start_year, seed=i, sbp_payments_path=sbp_payments_path)
    for y in range(start_year, stop_year):
        model.step()
        print("Run n° " + str(i+1) + ": year " + str(y) + " step completed.")
    collect_data_run(model, i)

Predicted a negative adoption of: [-0.000555]
Predicted a negative adoption of: [-0.000851]
Predicted a negative adoption of: [-0.000161]
Predicted a negative adoption of: [-0.001150]
Predicted a negative adoption of: [-0.000757]
Predicted a negative adoption of: [-0.000023]
Predicted a negative adoption of: [-0.000742]
Predicted a negative adoption of: [-0.000903]
Predicted a negative adoption of: [-0.000605]
Predicted a negative adoption of: [-0.000852]
Predicted a negative adoption of: [-0.000289]
Predicted a negative adoption of: [-0.000346]
Predicted a negative adoption of: [-0.000739]
Predicted a negative adoption of: [-0.000690]
Predicted a negative adoption of: [-0.000252]
Predicted a negative adoption of: [-0.000807]
Predicted a negative adoption of: [-0.000164]
Predicted a negative adoption of: [-0.000787]
Predicted a negative adoption of: [-0.000306]
Predicted a negative adoption of: [-0.000030]
Predicted a negative adoption of: [-0.000829]
Predicted a negative adoption of: 

Predicted a negative adoption of: [-0.002294]
Predicted a negative adoption of: [-0.001265]
Predicted a negative adoption of: [-0.000037]
Predicted a negative adoption of: [-0.001392]
Predicted a negative adoption of: [-0.001587]
Predicted a negative adoption of: [-0.004044]
Predicted a negative adoption of: [-0.000753]
Predicted a negative adoption of: [-0.000867]
Run n° 1: year 2012 step completed.
Predicted a negative adoption of: [-0.003251]
Predicted a negative adoption of: [-0.003399]
Predicted a negative adoption of: [-0.003859]
Predicted a negative adoption of: [-0.006245]
Predicted a negative adoption of: [-0.002672]
Predicted a negative adoption of: [-0.002525]
Predicted a negative adoption of: [-0.000483]
Predicted a negative adoption of: [-0.002090]
Predicted a negative adoption of: [-0.002343]
Predicted a negative adoption of: [-0.005264]
Predicted a negative adoption of: [-0.000199]
Predicted a negative adoption of: [-0.001320]
Predicted a negative adoption of: [-0.001826

Run n° 2: year 1997 step completed.
Predicted a negative adoption of: [-0.000097]
Predicted a negative adoption of: [-0.000037]
Predicted a negative adoption of: [-0.000057]
Predicted a negative adoption of: [-0.001137]
Predicted a negative adoption of: [-0.001128]
Predicted a negative adoption of: [-0.001140]
Predicted a negative adoption of: [-0.000031]
Predicted a negative adoption of: [-0.000870]
Predicted a negative adoption of: [-0.000794]
Predicted a negative adoption of: [-0.000854]
Predicted a negative adoption of: [-0.000576]
Predicted a negative adoption of: [-0.000321]
Predicted a negative adoption of: [-0.000727]
Predicted a negative adoption of: [-0.000135]
Predicted a negative adoption of: [-0.000720]
Predicted a negative adoption of: [-0.000183]
Predicted a negative adoption of: [-0.000291]
Run n° 2: year 1998 step completed.
Predicted a negative adoption of: [-0.000315]
Predicted a negative adoption of: [-0.000281]
Predicted a negative adoption of: [-0.001109]
Predicte

Predicted a negative adoption of: [-0.002614]
Predicted a negative adoption of: [-0.003651]
Predicted a negative adoption of: [-0.004916]
Predicted a negative adoption of: [-0.000587]
Predicted a negative adoption of: [-0.001784]
Predicted a negative adoption of: [-0.001909]
Predicted a negative adoption of: [-0.000582]
Predicted a negative adoption of: [-0.002927]
Predicted a negative adoption of: [-0.000264]
Predicted a negative adoption of: [-0.004471]
Run n° 2: year 2016 step completed.
Predicted a negative adoption of: [-0.000260]
Predicted a negative adoption of: [-0.006788]
Predicted a negative adoption of: [-0.005849]
Predicted a negative adoption of: [-0.000166]
Predicted a negative adoption of: [-0.002202]
Predicted a negative adoption of: [-0.007515]
Predicted a negative adoption of: [-0.002398]
Predicted a negative adoption of: [-0.003899]
Predicted a negative adoption of: [-0.005766]
Predicted a negative adoption of: [-0.001067]
Predicted a negative adoption of: [-0.000983

C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


...

C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Predicted a negative adoption of: [-0.000555]
Predicted a negative adoption of: [-0.000851]
Predicted a negative adoption of: [-0.000287]
Predicted a negative adoption of: [-0.000021]
Predicted a negative adoption of: [-0.001150]
Predicted a negative adoption of: [-0.000757]
Predicted a negative adoption of: [-0.000023]
Predicted a negative adoption of: [-0.000742]
Predicted a negative adoption of: [-0.000605]
Predicted a negative adoption of: [-0.000852]
Predicted a negative adoption of: [-0.000346]
Predicted a negative adoption of: [-0.001114]
Predicted a negative adoption of: [-0.000739]
Predicted a negative adoption of: [-0.000252]
Predicted a negative adoption of: [-0.000807]
Predicted a negative adoption of: [-0.000592]
Predicted a negative adoption of: [-0.000164]
Predicted a negative adoption of: [-0.000787]
Predicted a negative adoption of: [-0.000306]
Predicted a negative adoption of: [-0.000030]
Predicted a negative adoption of: [-0.000119]
Predicted a negative adoption of: 

Predicted a negative adoption of: [-0.002294]
Predicted a negative adoption of: [-0.001265]
Predicted a negative adoption of: [-0.000037]
Predicted a negative adoption of: [-0.001392]
Predicted a negative adoption of: [-0.001587]
Predicted a negative adoption of: [-0.000461]
Predicted a negative adoption of: [-0.009886]
Predicted a negative adoption of: [-0.000753]
Predicted a negative adoption of: [-0.000867]
Run n° 25: year 2012 step completed.
Predicted a negative adoption of: [-0.003399]
Predicted a negative adoption of: [-0.003859]
Predicted a negative adoption of: [-0.006245]
Predicted a negative adoption of: [-0.002672]
Predicted a negative adoption of: [-0.002525]
Predicted a negative adoption of: [-0.000483]
Predicted a negative adoption of: [-0.002090]
Predicted a negative adoption of: [-0.002343]
Predicted a negative adoption of: [-0.005264]
Predicted a negative adoption of: [-0.000423]
Predicted a negative adoption of: [-0.000199]
Predicted a negative adoption of: [-0.00132

C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Wall time: 28min 3s


In [17]:
munic_yearly_adoption.head(10)

Run 1    Run 2    Run 3    Run 4    Run 5    Run 6  \
Municipality Year                                                         
Évora        1996 0.001671 0.001671 0.000000 0.001671 0.001671 0.001671   
             1997 0.000000 0.001791 0.001791 0.001791 0.001791 0.001791   
             1998 0.001993 0.001993 0.001993 0.001993 0.001993 0.001993   
             1999 0.002269 0.002269 0.002269 0.002269 0.002269 0.002269   
             2000 0.002591 0.002591 0.002591 0.002591 0.002591 0.002591   
             2001 0.002561 0.002561 0.002561 0.002561 0.002561 0.002561   
             2002 0.002516 0.002516 0.002516 0.002516 0.002516 0.002516   
             2003 0.002452 0.002452 0.002452 0.002452 0.002452 0.002452   
             2004 0.002358 0.002358 0.002358 0.002358 0.002358 0.002358   
             2005 0.002224 0.002224 0.002224 0.002224 0.002224 0.002224   

                     Run 7    Run 8    Run 9   Run 10  ...   Run 16   Run 17  \
Municipality Year                                      ...                     
Évora        1996 0.001671 0.001671 0.001671 0.001671  ... 0.000000 0.001671   
             1997 0.001791 0.001791 0.001791 0.001791  ... 0.001791 0.001791   
             1998 0.001993 0.001993 0.001993 0.001993  ... 0.001993 0.001993   
             1999 0.002269 0.002269 0.002269 0.002269  ... 0.002269 0.002269   
             2000 0.002591 0.002591 0.002591 0.002591  ... 0.002591 0.000000   
             2001 0.002561 0.002561 0.002561 0.002561  ... 0.002561 0.002561   
             2002 0.002516 0.002516 0.002516 0.002516  ... 0.002516 0.002516   
             2003 0.002452 0.002452 0.002452 0.002452  ... 0.002452 0.002452   
             2004 0.002358 0.002358 0.002358 0.002358  ... 0.002358 0.002358   
             2005 0.002224 0.002224 0.002224 0.000000  ... 0.002224 0.002224   

                    Run 18   Run 19   Run 20   Run 21   Run 22   Run 23  \
Municipality Year                                                         
Évora        1996 0.001671 0.001671 0.001671 0.001671 0.001671 0.000000   
             1997 0.001791 0.001791 0.001791 0.001791 0.001791 0.001791   
             1998 0.001993 0.001993 0.001993 0.001993 0.001993 0.001993   
             1999 0.002269 0.002269 0.002269 0.002269 0.002269 0.000000   
             2000 0.002591 0.002591 0.002591 0.002591 0.002591 0.002591   
             2001 0.002561 0.002561 0.002561 0.002561 0.002561 0.002561   
             2002 0.002516 0.002516 0.002516 0.002516 0.002516 0.002516   
             2003 0.002452 0.002452 0.002452 0.002452 0.002452 0.002452   
             2004 0.002358 0.002358 0.002358 0.002358 0.002358 0.002358   
             2005 0.002224 0.002224 0.002224 0.002224 0.002224 0.002224   

                    Run 24   Run 25  
Municipality Year                    
Évora        1996 0.001671 0.001671  
             1997 0.001791 0.001791  
             1998 0.001993 0.001993  
             1999 0.002269 0.002269  
             2000 0.002591 0.002591  
             2001 0.002561 0.002561  
             2002 0.002516 0.002516  
             2003 0.002452 0.002452  
             2004 0.002358 0.002358  
             2005 0.002224 0.002224  

[10 rows x 25 columns]

## Save model output

In [18]:
os.mkdir("model_validation/output/output")
munic_yearly_adoption.to_csv("model_validation/output/output/municipalities_yearly_adoption.csv")
yearly_adoption_portugal_pred.to_csv("model_validation/output/output/portugal_yearly_adoption.csv")
cumulative_adoption_portugal_pred.to_csv("model_validation/output/output/portugal_cumulative_adoption.csv")

## Create output to plot map

In [19]:
import geopandas as gpd

In [20]:
def collect_agent_level_for_map(abm_model, year):
    """
    Function to collect cumulative adoption in a certain year (in ha and in fraction over pastures area)
    and the shape of the municialities in order to plot the map.
    
    Returns
    -------
    munic_cumul_adoption_and_shape : pd DataFrame
        DataFrame reporting for each muninicipality the total cumulative adoption in ha and its shape
    """
    municipality_agents = abm_model.schedule.agents
    #municipalities = [munic.Municipality for munic in municipality_agents]
    
    #munic_cumul_adoption_and_shape = pd.DataFrame(index=municipalities)
    munic_cumul_adoption_and_shape = gpd.GeoDataFrame()
    munic_cumul_adoption_and_shape['Total cumulative adoption [ha]'] = None
    munic_cumul_adoption_and_shape['Total cumulative adoption [fraction]'] = None
    munic_cumul_adoption_and_shape['geometry'] = None
    
    for munic in municipality_agents:
        yearly_adoption_until_year_ha = [v for k, v in munic.yearly_adoption_ha.items()
                                        if k <= year]
        munic_tot_cumul_adoption_year_ha = sum(yearly_adoption_until_year_ha)
        
        yearly_adoption_until_year = [v for k, v in munic.yearly_adoption.items()
                                     if k <= year]
        munic_tot_cumul_adoption_year = sum(yearly_adoption_until_year)
        
        munic_cumul_adoption_and_shape.loc[munic.Municipality] = (
            munic_tot_cumul_adoption_year_ha, 
            munic_tot_cumul_adoption_year,
            munic.shape
        )
    
    return munic_cumul_adoption_and_shape

In [21]:
year_for_map = 2012

In [22]:
pred_for_map = collect_agent_level_for_map(model, year_for_map)

In [23]:
pred_for_map.to_csv("model_validation/output/output/adoption_pred_for_map.csv")